In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
from sklearn.model_selection import train_test_split
import numpy as np
import os

# Загрузка данных из pickle-файлов
with open("X.pickle", "rb") as pickle_in:
    X = pickle.load(pickle_in)

with open("y.pickle", "rb") as pickle_in:
    y = pickle.load(pickle_in)

# Нормализация данных
X = X / 255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            # Создание модели
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for _ in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            # Формирование пути для логов
            log_dir = os.path.join("logs", NAME)
            os.makedirs(log_dir, exist_ok=True)
            tensorboard = TensorBoard(log_dir=log_dir)

            # Компиляция модели
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])

            # Разделение данных на обучающий, валидационный и тестовый наборы
            X_train, X_temp, y_train, y_temp = train_test_split(np.array(X), np.array(y), test_size=0.3, random_state=42)
            X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

            # Обучение модели с валидацией
            model.fit(X_train, y_train,
                      batch_size=32,
                      epochs=10,
                      validation_data=(X_val, y_val),
                      callbacks=[tensorboard])

            # Оценка точности модели на тестовом наборе данных
            test_loss, test_acc = model.evaluate(X_test, y_test)
            print('Test accuracy:', test_acc)

            # Сохранение модели
            model.save('64x3-CNN.keras')


3-conv-64-nodes-0-dense-1716006884


C:\Users\kumin\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


FailedPreconditionError: {{function_node __wrapped__CreateSummaryFileWriter_device_/job:localhost/replica:0/task:0/device:CPU:0}} logs\3-conv-64-nodes-0-dense-1716006884 is not a directory [Op:CreateSummaryFileWriter] name: 